# Workflows de Análise de Dados e Geoestatística

## Módulo 2 - Análise Exploratória de Dados com Python

### Imports

In [1]:
# imports

import os

import pandas as pd

# data viz
import plotly
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# configuring plots of seaborn
sns.set_style("white", {'axes.grid':False}, )

### Helper functions

In [3]:
# directories
def get_directories():
    # getting working paths
    CURRENT_DIR = os.path.dirname(os.path.realpath('__file__'))
    PARENT_DIR = os.path.dirname(CURRENT_DIR)
    #QUERY_DIR = os.path.join(CURRENT_DIR, 'queries')
    DATA_DIR = os.path.join(PARENT_DIR, 'data')
    
    dirs = {
        "CURRENT_DIR" : CURRENT_DIR,
        "PARENT_DIR" : PARENT_DIR,
        #"QUERY_DIR" : QUERY_DIR,
        "DATA_DIR" : DATA_DIR
    }

    return (dirs)


# metadados sobre dataframe
def meta_df(df):
    return pd.DataFrame({'dtypes': df.dtypes,
                         'nmissing': df.isna().sum(),
                         'missing%': round(df.isna().sum()/df.shape[0]*100, 2),
                         'nunique': df.nunique(),
                         'nunique%': round(df.nunique()/df.shape[0]*100, 2),
                         'size': df.shape[0]}, index = df.columns)

In [4]:
get_directories()

{'CURRENT_DIR': 'C:\\Users\\lucas\\Documents\\code\\workflows-geoestatistica-data\\modulo_2',
 'PARENT_DIR': 'C:\\Users\\lucas\\Documents\\code\\workflows-geoestatistica-data',
 'DATA_DIR': 'C:\\Users\\lucas\\Documents\\code\\workflows-geoestatistica-data\\data'}

In [5]:
# importing data

data_dir = get_directories()["DATA_DIR"]
df_jura = pd.read_csv(os.path.join(data_dir,"silver_jura.csv"))
df_jura.head(10)

,vlLocX,vlLocY,descLanduse,descUnit,vlCadmium,vlCobalt,vlChromium,vlNickel,vlLead,vlZinc,status,rankCadmium,rankCobalt,rankChromium,rankNickel,rankPb,rankZn,descAnomalyZn
0,1.932,1.004,campo,argoviano,0.135,4.520,15.08,5.24,37.00,32.56,NaN,0.00,0.14,0.02,0.03,0.28,0.04,normal
1,1.824,0.999,campo,argoviano,0.165,3.752,18.60,7.08,56.40,44.00,NaN,0.00,0.07,0.06,0.07,0.68,0.12,normal
2,1.883,1.176,campo,argoviano,0.195,3.920,21.80,7.52,49.60,43.20,NaN,0.01,0.10,0.12,0.10,0.56,0.10,normal
3,1.837,1.037,campo,argoviano,0.215,3.880,23.00,8.16,46.80,44.00,NaN,0.01,0.09,0.14,0.11,0.51,0.12,normal
4,3.768,2.775,floresta,kimmeridgiano,0.220,10.800,26.32,17.52,31.88,45.20,NaN,0.02,0.60,0.20,0.37,0.11,0.14,normal
5,1.839,0.995,campo,argoviano,0.225,4.240,24.88,10.12,55.60,56.00,NaN,0.02,0.12,0.17,0.16,0.68,0.26,normal
6,2.537,3.768,campo,argoviano,0.230,3.792,18.20,5.68,26.84,32.08,NaN,0.02,0.08,0.04,0.05,0.04,0.03,normal
7,2.502,3.988,campo,argoviano,0.240,3.760,18.92,5.20,22.36,32.24,NaN,0.03,0.08,0.06,0.03,0.02,0.04,normal
8,1.842,0.989,campo,argoviano,0.240,4.520,27.96,11.32,52.40,56.40,NaN,0.03,0.14,0.26,0.20,0.62,0.26,normal
9,1.897,1.220,campo,argoviano,0.260,3.480,16.24,4.76,27.00,27.20,NaN,0.04,0.04,0.03,0.01,0.05,0.01,normal


In [6]:
meta_df(df_jura)

,dtypes,nmissing,missing%,nunique,nunique%,size
vlLocX,float64,0,0.00,238,94.82,251
vlLocY,float64,0,0.00,241,96.02,251
descLanduse,object,0,0.00,4,1.59,251
descUnit,object,0,0.00,5,1.99,251
vlCadmium,float64,0,0.00,209,83.27,251
vlCobalt,float64,0,0.00,172,68.53,251
vlChromium,float64,0,0.00,198,78.88,251
vlNickel,float64,0,0.00,213,84.86,251
vlLead,float64,0,0.00,192,76.49,251
vlZinc,float64,0,0.00,183,72.91,251


In [7]:
df_jura.columns

Index(['vlLocX', 'vlLocY', 'descLanduse', 'descUnit', 'vlCadmium', 'vlCobalt',
       'vlChromium', 'vlNickel', 'vlLead', 'vlZinc', 'status', 'rankCadmium',
       'rankCobalt', 'rankChromium', 'rankNickel', 'rankPb', 'rankZn',
       'descAnomalyZn'],
      dtype='object')

In [8]:
grade_features = ["vlCadmium", "vlCobalt", "vlChromium", "vlNickel", "vlLead", "vlZinc", "descUnit"]
grade_features

['vlCadmium',
 'vlCobalt',
 'vlChromium',
 'vlNickel',
 'vlLead',
 'vlZinc',
 'descUnit']

### Visualizing Data

In [9]:
fig = px.scatter(
    x=df_jura["vlCadmium"],
    y=df_jura["vlZinc"], 
    color=df_jura['descUnit'],
    title="Scatter Cd X Zn"
  )

fig.update_layout(width=800, height=500, title_x=0.5,template="simple_white",font_family="Calibri", margin=dict(r=20,b=20)
)
fig.update_xaxes(showline=True,linewidth=0.5,linecolor='black',title_text='x',ticks="inside",tickwidth=0.5)
fig.update_yaxes(showline=True,linewidth=0.5,linecolor='black',title_text='y',ticks="inside",tickwidth=0.5)
fig.show()

#### Creating linear models

In [10]:
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.graph_objects as go

In [11]:
def plot_linear_model_pair (df, x_elem, y_elem):

  # objeto Regressor
  reg = LinearRegression()

  xm = df[x_elem].values[:,np.newaxis]
  ym = df[y_elem].values

  reg.fit(X=xm, y=ym)
  line = reg.predict(xm)

  print(xm)

  # plotting
  plt.figure(figsize=(12,8))
  sns.scatterplot(x=xm, y=ym)
  sns.lineplot(xm,line)
  plt.show()

  return reg.get_params()

In [12]:
plot_linear_model_pair(df_jura, "vlCadmium", "vlZinc")

[[0.135]
 [0.165]
 [0.195]
 [0.215]
 [0.22 ]
 [0.225]
 [0.23 ]
 [0.24 ]
 [0.24 ]
 [0.26 ]
 [0.275]
 [0.275]
 [0.28 ]
 [0.285]
 [0.3  ]
 [0.305]
 [0.32 ]
 [0.32 ]
 [0.325]
 [0.325]
 [0.35 ]
 [0.357]
 [0.36 ]
 [0.37 ]
 [0.375]
 [0.4  ]
 [0.4  ]
 [0.425]
 [0.425]
 [0.44 ]
 [0.45 ]
 [0.455]
 [0.465]
 [0.485]
 [0.49 ]
 [0.495]
 [0.505]
 [0.515]
 [0.515]
 [0.525]
 [0.525]
 [0.535]
 [0.54 ]
 [0.55 ]
 [0.552]
 [0.555]
 [0.56 ]
 [0.56 ]
 [0.56 ]
 [0.57 ]
 [0.575]
 [0.58 ]
 [0.585]
 [0.605]
 [0.605]
 [0.61 ]
 [0.62 ]
 [0.62 ]
 [0.625]
 [0.625]
 [0.625]
 [0.63 ]
 [0.63 ]
 [0.645]
 [0.65 ]
 [0.655]
 [0.685]
 [0.685]
 [0.69 ]
 [0.69 ]
 [0.705]
 [0.705]
 [0.72 ]
 [0.725]
 [0.733]
 [0.735]
 [0.74 ]
 [0.74 ]
 [0.75 ]
 [0.765]
 [0.765]
 [0.77 ]
 [0.775]
 [0.785]
 [0.795]
 [0.8  ]
 [0.805]
 [0.81 ]
 [0.81 ]
 [0.82 ]
 [0.83 ]
 [0.84 ]
 [0.844]
 [0.845]
 [0.849]
 [0.855]
 [0.86 ]
 [0.86 ]
 [0.86 ]
 [0.875]
 [0.885]
 [0.89 ]
 [0.894]
 [0.895]
 [0.9  ]
 [0.905]
 [0.91 ]
 [0.93 ]
 [0.93 ]
 [0.945]
 [0.95 ]
 

ValueError: Per-column arrays must each be 1-dimensional

<Figure size 864x576 with 0 Axes>

In [ ]:
grade_features.remove('descUnit')
grade_features

['vlCadmium', 'vlCobalt', 'vlChromium', 'vlNickel', 'vlLead', 'vlZinc']

In [ ]:
def linear_model_multiple (df, grade_features, y_elem):
  
  # Regressor
  reg = LinearRegression()
  
  grade_x = grade_features.copy()
  grade_x.remove(y_elem)

  xm = df[grade_x].values
  ym = df[y_elem].values

  reg.fit(X=xm, y=ym)
  score = reg.score(X=xm, y=ym)
  y_pred = reg.predict(xm)

  model_dict = {
    'model': reg,
    'element':y_elem,
    'score': score, 
    'pred': y_pred,
    'params': reg.get_params()
  }

  return model_dict

model = linear_model_multiple(df_jura, grade_features, 'vlNickel')
model

{'model': LinearRegression(),
 'element': 'vlNickel',
 'score': 0.7379665282676895,
 'pred': array([ 6.68413626,  7.16045423,  8.10315763,  8.35113888, 17.53168175,
         9.62014989,  6.51113883,  6.59962391, 10.73211176,  5.56934829,
         5.65545421,  6.65877691, 14.20365434, 19.81615526,  6.98652154,
         6.97595242,  7.01041014,  9.29597322,  6.69644477,  5.95625363,
         8.1708807 ,  7.6468843 , 12.06032749,  7.25054516, 15.24014418,
         8.15990672, 16.91517565,  7.67402964,  9.361945  , 13.10845346,
         8.54165114,  5.4933088 ,  9.27183828, 16.01637316, 15.93011308,
        17.34681799, 13.37890977, 10.37983782, 12.38689242,  8.14569711,
        22.26956457,  8.4619326 , 13.89180356, 17.56341374, 24.32281599,
        17.91138602, 12.01851752, 15.29106661, 19.23786586, 11.24897607,
        17.36500323, 13.92288978, 21.01292095, 12.16637871, 17.48280995,
        11.75615758, 12.5192524 , 23.61603592,  8.38649946, 15.05051891,
        21.91384081,  8.25728838

In [ ]:
for a in grade_features:
  print(plot_linear_model_multiple(df_jura,a))

NameError: name 'plot_linear_model_multiple' is not defined